In [298]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv("mazimac.csv",encoding = 'euc-kr')

In [12]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
null_data = df['입지구분'].isnull()
null_data
# 조건를 충족하는 데이터를 필터링하여 새로운 변수에 저장합니다.
nantent = df[null_data]

In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

driver = webdriver.Chrome("chromedriver")
driver.implicitly_wait(5)
driver.get("https://map.naver.com/v5/search")
list =[] #link넣기
excepted = [] #예외 야영장 이름 넣기

# 검색창에 검색어 입력하기
size = nantent.shape[0]
for i in range(size) :
    search_box = driver.find_element_by_css_selector("div.input_box>input.input_search")
    search_box.clear()
    search_box.send_keys(nantent['야영장명'].iloc[i])
# nantent['야영장명'].iloc[i]

    # 검색버튼 누르기
    search_box.send_keys(Keys.ENTER)
    
    #검색 리스트 중 맨위 누르기
    time.sleep(2)

    try:
        driver.switch_to.frame("searchIframe")
        slist = driver.find_element_by_css_selector("#app-root > div > div._2lx2y")
        #캠핑장 이름이 잘못되면 그냥 넣어주자 리스트
        if slist.text == "조건에 맞는 업체가 없습니다." :
            excepted.append(nantent['야영장명'].iloc[i])
            list.append([])
            driver.switch_to_default_content()
            continue

        searchlist = driver.find_elements_by_css_selector("#_pcmap_list_scroll_container > ul > li")
        flag = False
        
        if len(searchlist) > 1 : #결과값이 하나이상이면
            for search in searchlist :
                try:
                    result = search.find_element_by_css_selector("div._7jQRv > div._1uXIN > a")
                    address = search.find_element_by_css_selector("div._7jQRv > div.qGsds > span > a > span._3W_ec")
                except:
                    result = search.find_element_by_css_selector("a > div.YQSTs > div > span")
                    address = search.find_element_by_css_selector("a > div._2yrPK")
                    
                adr = nantent['주소'].iloc[i].split(' ')
                si = adr[1][: -1]
                gu = adr[2]
                
                if(si in address.text and gu in address.text):       
                    flag = True
                    break
                 
                
        else : #결과값이 하나
            result = searchlist[0].find_element_by_css_selector("a")
            address = searchlist[0].find_element_by_css_selector("div._2yrPK")
            adr = nantent['주소'].iloc[i].split(' ')
            si = adr[1][: -1]
            gu = adr[2]
            if(si in address.text and gu in address.text):
                flag = True

        if flag == True : 
            result.click()
        else :
            driver.switch_to_default_content()
            list.append([])
            continue
        
    except: #바로 블로그 리뷰가 나온다면
        trash = 0

    
#     #장소 클릭 후 블로그 리뷰 클릭
    driver.switch_to_default_content()
    driver.switch_to.frame("entryIframe")
    blogreviewbutton = driver.find_elements_by_css_selector("#app-root > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR._2z4r0 > div > span._1Y6hi")
    
    time.sleep(2)
    blogreview = False
    for button in blogreviewbutton :
        if '블로그' in button.text :
            blogreview = True
            button.find_element_by_css_selector("a").click()
#     if blogreview == False :
#         driver.switch_to_default_content()
#         list.append([])
#         continue
        
    #블로그 리뷰 하나씩 확인
    time.sleep(5)
    blogs = driver.find_elements_by_css_selector("#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div:nth-child(4) > div:nth-child(2) > div.place_section_content > ul > li")
    
    if len(blogs) == 0 :
        blogs = driver.find_elements_by_css_selector("#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > ul > li")
        
        if len(blogs) == 0 :
            blogs = driver.find_elements_by_css_selector("#app-root > div > div.place_detail_wrapper > div:nth-child(4) > div > div:nth-child(2) > div.place_section_content > ul > li")
    
    addblog = []
    for blog in blogs:
        link = blog.find_element_by_css_selector("a > div._32TVI > span._1Z2mH")

        if '블로그' in link.text :
            addblog.append(blog.find_element_by_css_selector("a").get_attribute("href"))
    list.append(addblog)
    driver.switch_to_default_content()
    
    
driver.close()

<ipython-input-14-6a62516f1e2c>:78: DeprecationWarning: use driver.switch_to.default_content instead
  driver.switch_to_default_content()
<ipython-input-14-6a62516f1e2c>:110: DeprecationWarning: use driver.switch_to.default_content instead
  driver.switch_to_default_content()
<ipython-input-14-6a62516f1e2c>:69: DeprecationWarning: use driver.switch_to.default_content instead
  driver.switch_to_default_content()


In [15]:
cnt = 0
campingreview = []
for i in range(len(list)) :#리스트 만큼
    articles = []
    if(len(list[i]) != 0):#리뷰가 존재하다면
        for j in list[i]:#하나하나 들어간다.
            driver = webdriver.Chrome("chromedriver")
            driver.implicitly_wait(5)
            driver.get(j)
            driver.switch_to.frame("mainFrame")
            try:
                review = driver.find_element_by_css_selector('#printPost1 > tbody > tr > td.bcc')
            except :
                print(j)
            articles.append(review.text)
            driver.close()
    campingreview.append(articles)
    

In [192]:
list[0]

TypeError: 'set' object is not subscriptable

In [18]:
#한글 형태소 분석
# pip install --upgrade pip
# pip install konlpy
# pip install JPype1-1.2.0-cp38-cp38-win_amd64.whl

In [74]:
from konlpy.tag import Kkma,Okt

kkma = Kkma()
sentence = "오늘의 날짜는 9월 2일이다."

kkma.morphs(sentence)
kkma.nouns(sentence)
# kkma.pos(sentence)

['오늘', '날짜', '9', '9월', '월', '2', '2일', '일']

In [2]:
#영화리뷰로 학습 데이터 만들기

def read_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

train_data = read_data('ratings_train.txt')
test_data = read_data('ratings_test.txt')

In [3]:
import json
import os
from pprint import pprint
from konlpy.tag import Okt

okt = Okt()

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

if os.path.isfile('train_docs.json'):
    with open('train_docs.json') as f:
        train_docs = json.load(f)
    with open('test_docs.json') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
    # JSON 파일로 저장
    with open('train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

# 예쁘게(?) 출력하기 위해서 pprint 라이브러리 사용
pprint(train_docs[0])

(['아/Exclamation',
  '더빙/Noun',
  '../Punctuation',
  '진짜/Noun',
  '짜증나다/Adjective',
  '목소리/Noun'],
 '0')


In [4]:
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

2159924


In [5]:
import nltk
text = nltk.Text(tokens, name='NMSC')

# 전체 토큰의 개수
print(len(text.tokens))

# 중복을 제외한 토큰의 개수
print(len(set(text.tokens)))            

# 출현 빈도가 높은 상위 토큰 10개
pprint(text.vocab().most_common(10))


2159924
49895
[('./Punctuation', 67777),
 ('영화/Noun', 50818),
 ('하다/Verb', 41209),
 ('이/Josa', 38540),
 ('보다/Verb', 38538),
 ('의/Josa', 30188),
 ('../Punctuation', 29055),
 ('가/Josa', 26627),
 ('에/Josa', 26468),
 ('을/Josa', 23118)]


In [6]:
# 시간이 꽤 걸립니다! 시간을 절약하고 싶으면 most_common의 매개변수를 줄여보세요.
selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [7]:
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [8]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

Epoch 1/10
293/293 [==============================] - 18s 58ms/step - loss: 0.4486 - binary_accuracy: 0.8012 1s - loss: 0.4544 
Epoch 2/10
293/293 [==============================] - 11s 38ms/step - loss: 0.3118 - binary_accuracy: 0.8676
Epoch 3/10
293/293 [==============================] - 6s 21ms/step - loss: 0.2850 - binary_accuracy: 0.8816
Epoch 4/10
293/293 [==============================] - 5s 18ms/step - loss: 0.2665 - binary_accuracy: 0.8916
Epoch 5/10
293/293 [==============================] - 5s 17ms/step - loss: 0.2437 - binary_accuracy: 0.9035
Epoch 6/10
293/293 [==============================] - 7s 25ms/step - loss: 0.2217 - binary_accuracy: 0.9139
Epoch 7/10
293/293 [==============================] - 9s 30ms/step - loss: 0.2060 - binary_accuracy: 0.9207
Epoch 8/10
293/293 [==============================] - 10s 33ms/step - loss: 0.1872 - binary_accuracy: 0.9286
Epoch 9/10
293/293 [==============================] - 9s 30ms/step - loss: 0.1724 - binary_accuracy: 0.9345
Epoch 

In [261]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score >= 0.5):
        return True

In [260]:
predict_pos_neg("가족과 오면 좋아요")
predict_pos_neg("주말은 예약하기 힘들어서")
predict_pos_neg("사이트 전기 안되고 1만원")
predict_pos_neg("원래 골프장으로 만들어서 필드가 아주 예쁨")
predict_pos_neg("노을 캠핑장은 조용히 도란도란 즐기기 좋았다")

[가족과 오면 좋아요]는 92.34% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^

[주말은 예약하기 힘들어서]는 70.41% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[사이트 전기 안되고 1만원]는 93.91% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

[원래 골프장으로 만들어서 필드가 아주 예쁨]는 55.97% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^

[노을 캠핑장은 조용히 도란도란 즐기기 좋았다]는 94.55% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^



In [21]:
import re
import os
import sys
import json

from pykospacing import spacing
from konlpy.tag import Kkma
from konlpy.tag import Okt
import soynlp


kkma = Kkma()
okt = Okt()

In [267]:
locationkeys = location.keys()

def Morphemeanalysis(sentence): #형태소 분석 후 입지 나누기
    nouns = okt.nouns(sentence)
    print(nouns)
    for key in locationkeys:
       for value in location[key]:
            if value in nouns :
                if predict_pos_neg(sentence) == True:
                    loca.add(key)

In [196]:
def Hashtaganalysis(sentence): #형태소 분석 후 입지 나누기
    
    for key in locationkeys:
       for value in location[key]:
            if value in sentence :
                loca.add(key)

In [319]:
location = {"도심" : ['도시','도심','건물','아파트'],
            '산' : ['산'],
            '숲' : ['숲','그늘','삼림','자연'],
            '섬' : ['섬'],
            '계곡' : ['계곡','줄기'],
            '강' : ['강'],
            '바다' : ['바다','해변','해수욕','밀물','썰물'],
            '호수' : ['호수','댐','저수지']}

r = 0
for camping in campingreview: #한 캠핑장에 대한 모든 블로그 리뷰
    loca = set([])
    
    for review in camping: #블로그 리뷰 하나씩 확인
        verbs = review.split('\n')
        for verb in verbs: #엔터로 나눈 문장
            sentences = verb.split('.')
            for sentence in sentences: #.로 나눈 문장
                sentence = re.sub('([a-zA-Z])','',sentence)
                sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
                sentence = re.sub('[-=+,/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)

                if('#' in sentence): #hashtag라면
                    hashtag = okt.pos(sentence, join = True)
                    for has in hashtag:
                        has = has.split('/')
                        if has[1] == 'Hashtag':
                            Hashtaganalysis(has[0][1:])
                            
    print(loca) #캠핑장 하나 끝나면 출력
    print(nantent.iloc[r]['야영장명'])
    locationlist = tuple(loca)
    imsi = ''
    for i in range(len(locationlist)):
        imsi +=locationlist[i]
        if i == len(locationlist) -1:
            continue
        imsi += ','
#     campingname = nantent.iloc[r]['야영장명']
#     try:
#         df.loc[(df.야영장명 == campingname),['입지구분']] = [imsi]
#     except:
#         print(campingname)
    
    r += 1

{'숲', '산'}
노을공원 노을캠핑장 
set()
녹두장군 전봉준관
{'산'}
바인밸리 수영장 캠핑장
set()
늘푸른캠핑장
{'숲', '산'}
다솔관광농원
{'산'}
다인캠핑장 
{'강'}
당평오토캠핑장 
{'산'}
팝콘야영장
{'계곡'}
대현캠핑장
{'숲'}
덕암호캠핑장 
{'산', '계곡'}
덕유대 자동차야영장 
{'강'}
덕포진누리마을캠핑장
{'산', '계곡'}
도둑골 야영장 
{'산', '계곡'}
도토리캠프 
{'바다', '산'}
돌고래 하우스 카라반
{'산'}
돌고래펜션야영장(일반) 
{'강', '바다', '계곡'}
돌집야영장
{'강', '숲', '산'}
동그라미캠핑장 
{'산'}
동방캠핑하우스 
{'숲', '산'}
동화힐링캠프 
{'숲', '도심', '산', '계곡'}
둥지펜션 캠핑장 
set()
룩스글램핑앤팜 
{'강', '숲', '산', '계곡'}
리스캐빈 
{'강', '숲', '산', '계곡'}
만산동 국민여가캠핑장
{'바다', '산'}
만석꾼 캠핑장 
{'숲', '바다', '산', '계곡'}
만수산 자연휴양림 
{'산', '계곡'}
묘솔 캠핑장 
{'산', '계곡'}
보보캠핑
{'산', '계곡'}
무주 구천동 캠핑장 
{'강', '호수'}
문수골 힐링캠핑장
{'강', '산'}
문암생태공원 
{'강', '숲', '산', '계곡'}
물가애 캠핑장
{'강', '숲', '산', '계곡'}
미산가족오토캠핑장 
{'숲', '산'}
민주지산 자연휴양림 
{'계곡', '산', '호수'}
밀양댐 스쿨캠핑장 
{'강'}
바람하늘공원 야영장 
{'숲', '산'}
박달재 자연휴양림캠핑장
set()
박달재옛길캠핑장 
{'숲', '산', '섬'}
반딧불자연캠핑장 
{'강', '산'}
밤벌오토캠핑장
{'숲', '계곡'}
방화동 가족휴가촌 오토캠핑장 
{'강', '산'}
백산금빛캠프장
{'강', '바다', '산'}
백운산체험농장 
{'바다'}
백일야영장 
{'산', '계곡'}
뱀사골자동차야영장
{'강', '호수'}
베이스캠프 캠핑장 
{'산', '호수'}
벽계야영장 

{'바다', '산', '계곡'}
가야산백운오토캠핑장


KeyboardInterrupt: 

In [313]:
df.loc[df['야영장명'] =='산여울캠핑장']
# "톱머리 글램핑장"

,컨텐츠ID,야영장명,한줄소개,소개,특징,입지구분,도,시군구,주소,주소상세,경도(X),위도(Y),전화,홈페이지,예약 페이지,예약 구분,오토,글램핑,카라반,툴팁,글램핑 - 내부시설,카라반 - 내부시설,운영기간,운영일,개인 트레일러 동반 여부,화로대,부대시설,부대시설 기타,주변이용가능시설,주변이용가능시설 기타,야외 화장실,야외 샤워실,자체문화행사 여부,자체문화행사명,체험프로그램 여부,체험프로그램명,테마환경,캠핑장비대여,애완동물출입,대표이미지
647,648,산여울캠핑장,NaN,NaN,NaN,계곡,강원도,영월군,강원 영월군 무릉도원면 백년계곡길 142,산여울캠핑장,128.271781,37.377076,033-375-2268,http://www.sanyaiul.com/,야놀자,NaN,N,N,Y,NaN,NaN,NaN,"봄,여름,가을",평일+주말,Y,개별,"전기,무선인터넷,장작판매,온수,운동시설",NaN,계곡 물놀이,NaN,Y,Y,N,NaN,N,NaN,NaN,NaN,가능(소형견),https://gocamping.or.kr/upload/camp/1503/thumb...


In [320]:
df.to_csv('test.csv')